In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import classification_report

In [3]:
X = pd.read_csv('../glass_formula_a.csv')

df = pd.read_csv('../glass.csv', sep=",")
Y = df['Type']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)
X

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,cos(RI+Na),...,cos(Si+K),cos(Si+Ca),cos(Si+Ba),cos(Si+Fe),cos(K+Ca),cos(K+Ba),cos(K+Fe),cos(Ca+Ba),cos(Ca+Fe),cos(Ba+Fe)
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,-0.854113,...,-0.914457,0.407201,-0.888546,-0.888546,-0.816902,0.998201,0.998201,-0.780846,-0.780846,1.000000
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,-0.955232,...,-0.578939,0.434414,-0.890037,-0.890037,-0.440377,0.886995,0.886995,0.023979,0.023979,1.000000
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,-0.788898,...,-0.432648,0.612633,-0.742924,-0.742924,-0.310785,0.924909,0.924909,0.073914,0.073914,1.000000
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,-0.556771,...,-0.603136,0.658924,-0.938212,-0.938212,-0.805203,0.841901,0.841901,-0.357900,-0.357900,1.000000
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,-0.605388,...,-0.196147,0.862094,-0.679754,-0.679754,-0.693271,0.852525,0.852525,-0.214342,-0.214342,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,-0.998662,...,-0.907556,0.994110,-0.156778,-0.938212,-0.986455,0.417595,0.996802,-0.685707,-0.970191,0.488872
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,-0.745916,...,-0.694286,0.975589,0.732899,-0.694286,-0.519289,-0.019202,1.000000,-0.844470,-0.519289,-0.019202
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,-0.985127,...,-0.396249,0.984095,0.943346,-0.396249,-0.553048,-0.069148,1.000000,-0.792913,-0.553048,-0.069148
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,-0.982278,...,-0.215718,0.917682,0.976284,-0.215718,-0.585923,0.000796,1.000000,-0.810833,-0.585923,0.000796


In [4]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV, RFE

# Logistic Regression

In [5]:
lr_rfecv = RFECV(estimator=LogisticRegression(), 
                  cv=10, 
                  min_features_to_select=10,
                scoring="accuracy")
lr_rfecv.fit(X, Y)

RFECV(cv=10, estimator=LogisticRegression(), min_features_to_select=10,
      scoring='accuracy')

In [6]:
feature_names = list(X.columns)
# to print out results
selected_features = []
for idx, name in enumerate(feature_names):
    if lr_rfecv.support_[idx]:
        selected_features.append(name)



In [7]:
x_lr = X_train[selected_features]
lr = LogisticRegression()
lr.fit(x_lr, y_train)

x_test_filtered = X_train[selected_features]
y_test_pred = lr.predict(x_test_filtered)
print(classification_report(y_train, y_test_pred))
print(cross_val_score(lr, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.68      0.86      0.76        58
           2       0.83      0.83      0.83        66
           3       0.00      0.00      0.00        15
           5       1.00      1.00      1.00        12
           6       1.00      1.00      1.00         6
           7       1.00      0.96      0.98        24

    accuracy                           0.81       181
   macro avg       0.75      0.78      0.76       181
weighted avg       0.75      0.81      0.78       181

0.6909356725146198


# SVM

In [8]:
from sklearn import svm
from sklearn.svm import LinearSVC

In [9]:
rf_svm_ = RFECV(estimator=LinearSVC(), 
                cv=10, 
                min_features_to_select=10,
                scoring="accuracy")
rf_svm_.fit(X, Y)

RFECV(cv=10, estimator=LinearSVC(), min_features_to_select=10,
      scoring='accuracy')

In [10]:
feature_names = list(X.columns)
# to print out results
selected_features = []
for idx, name in enumerate(feature_names):
    if rf_svm_.support_[idx]:
        selected_features.append(name)

selected_features

['Al',
 'Ba',
 'Fe',
 'cos(RI+Mg)',
 'cos(RI+Al)',
 'cos(RI+K)',
 'cos(RI+Ba)',
 'cos(RI+Fe)',
 'cos(Na+Mg)',
 'cos(Na+Ba)',
 'cos(Na+Fe)',
 'cos(Mg+Al)',
 'cos(Mg+K)',
 'cos(Mg+Ca)',
 'cos(Al+Si)',
 'cos(Al+K)',
 'cos(Si+K)',
 'cos(Si+Ba)',
 'cos(K+Ca)',
 'cos(Ca+Fe)']

In [11]:
x_lr = X_train[selected_features]
lr = svm.SVC()
lr.fit(x_lr, y_train)

x_test_filtered = X_train[selected_features]
y_test_pred = lr.predict(x_test_filtered)
print(classification_report(y_train, y_test_pred))
print(cross_val_score(lr, X_train, y_train, cv=10).mean())

              precision    recall  f1-score   support

           1       0.65      0.88      0.75        58
           2       0.80      0.80      0.80        66
           3       0.00      0.00      0.00        15
           5       0.92      1.00      0.96        12
           6       1.00      0.17      0.29         6
           7       1.00      0.96      0.98        24

    accuracy                           0.77       181
   macro avg       0.73      0.63      0.63       181
weighted avg       0.73      0.77      0.74       181

0.36461988304093573
